In [ ]:
import os
import geopandas as gpd
import rasterio
from rasterio import features
from rasterio.merge import merge
from rasterio.errors import RasterioIOError
from rasterio.features import geometry_mask
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from shapely.geometry import shape, mapping, box
import numpy as np
import matplotlib.pyplot as plt
import osmnx as ox

#from my_functions import merge_tiff_files, change_raster_values #this would be the ideal solution, but doesn't work yet

# functions

## combine tiff's


In [ ]:
def merge_tiff_files(tif_files, output_file):
    src_files_to_mosaic = []
    
    for file in tif_files:
        try:
            src = rasterio.open(file)
            src_files_to_mosaic.append(src)
        except RasterioIOError:
            print(f"Skipping non-existing file: {file}")
    
    mosaic, out_trans = merge(src_files_to_mosaic)
    
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": mosaic.shape[1],
                     "width": mosaic.shape[2],
                     "transform": out_trans,
                     "compress": 'lzw'})
    
    with rasterio.open(output_file, "w", **out_meta) as dest:
        dest.write(mosaic)
    
    for src in src_files_to_mosaic:
        src.close()

## change raster values (reclassify has more options)


In [ ]:
def change_raster_values(input_file, output_file, values, new_value):
    # Open the input TIFF file and read the first band as a NumPy array
    with rasterio.open(input_file) as src:
        raster = src.read(1)

        # Convert values to a list if a single integer is provided
        if not isinstance(values, list):
            values = [values]

        # Change pixel values in the raster array
        mask = np.isin(raster, values)
        raster = np.where(mask, new_value, raster)

        # Copy the metadata from the source dataset
        meta = src.meta.copy()

        # Update the metadata for the modified dataset
        meta.update(dtype=rasterio.int32, nodata=new_value, compress='lzw')

        # Save the modified raster array as a new TIFF file
        with rasterio.open(output_file, "w", **meta) as dest:
            dest.write(raster.astype(rasterio.int32), 1)

## change values outside mask to chosen value


In [ ]:
def change_values_outside_mask(input_file, output_file, mask, new_value = 0):
    # Load the input TIF file
    with rasterio.open(input_file) as src:
        # Read the input TIF file as an array
        data = src.read(1)

        # Extract the first geometry from the GeoDataFrame
        mask_geometry = shape(mask.iloc[0]['geometry'])

        # Create a mask using the input TIF file's georeferencing and the GDF mask
        mask_array = geometry_mask([mapping(mask_geometry)], src.shape, transform=src.transform, invert=True)

        # Replace values outside the mask with the new value
        data = np.where(mask_array, data, new_value)

        # Update the metadata and write the modified array to the output TIF file
        meta = src.meta
        meta.update(compress='lzw')
        with rasterio.open(output_file, 'w', **meta) as dst:
            dst.write(data, 1)

## clip to bbox


In [ ]:
def clip_tif_to_bbox(tif_path, gdf_boundary, output_path):
    # Open the TIFF file
    tif = rasterio.open(tif_path)

    # Extract the bounding box coordinates
    bounds = gdf_boundary.geometry.total_bounds
    bbox = box(*bounds)

    # Clip the TIFF file to the bounding box
    clipped, transform = mask(tif, [bbox], crop=True)

    # Update the metadata of the clipped TIFF
    clipped_meta = tif.meta.copy()
    clipped_meta.update({
        "height": clipped.shape[1],
        "width": clipped.shape[2],
        "transform": transform,
        "compress": 'lzw'
    })

    # Save the clipped TIFF to the output path
    with rasterio.open(output_path, "w", **clipped_meta) as output:
        output.write(clipped)

In [ ]:
def reproject_raster(epsg, input_file_path, output_file_path):
    #open source raster
    srcRst = rasterio.open(input_file_path)

    dstCrs = {'init': epsg}

    #calculate transform array and shape of reprojected raster
    transform, width, height = calculate_default_transform(
            srcRst.crs, dstCrs, srcRst.width, srcRst.height, *srcRst.bounds)

    #working of the meta for the destination raster
    kwargs = srcRst.meta.copy()
    kwargs.update({
            'crs': dstCrs,
            'transform': transform,
            'width': width,
            'height': height,
            'compress': 'lzw'
        })

    #open destination raster
    dstRst = rasterio.open(output_file_path, 'w', **kwargs)

    #reproject and save raster band data
    for i in range(1, srcRst.count + 1):
        reproject(
            source=rasterio.band(srcRst, i),
            destination=rasterio.band(dstRst, i),
            #src_transform=srcRst.transform,
            src_crs=srcRst.crs,
            #dst_transform=transform,
            dst_crs=dstCrs,
            resampling=Resampling.nearest)
    #close destination raster
    dstRst.close()

In [ ]:
def reclassify(input_file, output_file, reclassify_values):
    # Open the input TIFF file and read the first band as a NumPy array
    with rasterio.open(input_file) as src:
        raster = src.read(1)

        # Create a copy of the raster array to store the reclassified values
        reclassified_raster = np.copy(raster)

        # Loop through the reclassification dictionary and update the pixel values
        for new_value, values in reclassify_values.items():
            if isinstance(values, int):
                values = [values]
            mask = np.isin(raster, values)
            reclassified_raster[mask] = new_value

        # Copy the metadata from the source dataset
        meta = src.meta.copy()

        # Update the metadata for the modified dataset
        meta.update(dtype=rasterio.int32, compress='lzw')

        # Save the modified raster array as a new TIFF file
        with rasterio.open(output_file, "w", **meta) as dest:
            dest.write(reclassified_raster.astype(rasterio.int32), 1)

# import extent


In [ ]:
crs = "EPSG:4326"
gdf_boundary = ox.geocode_to_gdf("borneo").dropna(axis=1, how="any").to_crs(crs)
# Export the GeoDataFrame as a shapefile
gdf_boundary.to_file("../../07_data_prepared/boundary/borneo_boundary.shp")

# prepare oil palm data (2019)


In [ ]:
directory = '../../06_data'

oil_palm_2019 = gpd.read_file(os.path.join(directory, f"vegetation/oil_palm/High_resolution_global_industrial_and_smallholder_oil_palm_map_for_2019/grid/grid.shp"))

# Reproject oil_palm_2019 to match the CRS of gdf_boundary
oil_palm_2019 = oil_palm_2019.to_crs(gdf_boundary.crs)

oil_palm_2019_borneo = oil_palm_2019.clip(gdf_boundary)

# Extract the 'ID' column values as a list
id_values = oil_palm_2019_borneo['ID'].tolist()

# Convert the values to integers and then to strings
id_values = [str(int(value)) for value in id_values]

# List to store the TIFF file paths
tif_files = []
reclassified_tif_files = []

# Construct the TIFF file paths and reclassify the individual TIFF files

for value in id_values:
    tif_file = os.path.join(directory, f"vegetation/oil_palm/High_resolution_global_industrial_and_smallholder_oil_palm_map_for_2019/oil_palm_map/L2_2019b_{value}.tif")
    if os.path.exists(tif_file):
        tif_files.append(tif_file)
        path = os.path.join(directory, f"vegetation/oil_palm/High_resolution_global_industrial_and_smallholder_oil_palm_map_for_2019/oil_palm_reclasssified_borneo/L2_2019b_{value}.tif")
        reclassify_values = {0: 3}
        reclassify(tif_file, path, reclassify_values)
        clip_tif_to_bbox(path, gdf_boundary, path)
        change_values_outside_mask(path, path, gdf_boundary, 0)
        reproject_raster('EPSG:23846', path, os.path.join(directory, f"vegetation/oil_palm/High_resolution_global_industrial_and_smallholder_oil_palm_map_for_2019/oil_palm_reclasssified_borneo/utm_L2_2019b_{value}.tif"))
        reclassified_tif_files.append(os.path.join(directory, f"vegetation/oil_palm/High_resolution_global_industrial_and_smallholder_oil_palm_map_for_2019/oil_palm_reclasssified_borneo/utm_L2_2019b_{value}.tif"))
    else:
        print(f"File not found: {tif_file}")


# Merge the processed TIFF files
merge_tiff_files(reclassified_tif_files, '../../07_data_prepared/oil_palm/utm_oil_palm_2019_borneo.tif')


# prepare oil palm detection data


In [ ]:
tif_files = ['../../06_data/vegetation/oil_palm/year_of_detection/op_extent_age_N.tif', '../../06_data/vegetation/oil_palm/year_of_detection/op_extent_age_S.tif']

merge_tiff_files(tif_files, '../../07_data_prepared/oil_palm/detection_year.tif')

clip_tif_to_bbox('../../07_data_prepared/oil_palm/detection_year.tif', gdf_boundary, '../../07_data_prepared/oil_palm/detection_year.tif')

change_values_outside_mask('../../07_data_prepared/oil_palm/detection_year.tif', '../../07_data_prepared/oil_palm/detection_year.tif', gdf_boundary, 0)

# data information: The data values range from 0 to 37 where 0 is no data and subsequent numbers run through until 2017, the last year of our analysis. A value of 4 corresponds to the year 1984, the first year oil palm was detected and 37 corresponds to 2017. Values 1 to 3 are not present.
# --> therefore every yea

reclassify_values = {2000: list(range(1, 21)),
                     2001: 21,
                     2002: 22,
                     2003: 23,
                     2004: 24,
                     2005: 25,
                     2006: 26,
                     2007: 27,
                     2008: 28,
                     2009: 29,
                     2010: 30,
                     2011: 31,
                     2012: 32,
                     2013: 33,
                     2014: 34,
                     2015: 35,
                     2016: 36,
                     2017: 37}
reclassify('../../07_data_prepared/oil_palm/detection_year.tif', '../../07_data_prepared/oil_palm/detection_year_2000_2017.tif', reclassify_values)



# reproject to epsg:23846
reproject_raster('EPSG:23846', '../../07_data_prepared/oil_palm/detection_year_2000_2017.tif', '../../07_data_prepared/oil_palm/utm_detection_year_2000_2017.tif')

# prepare global forest watch data
## deforestation


In [ ]:
tif_files = ['../../06_data/deforestation/global_forest_watch/Hansen_GFC_2021_v1_9_lossyear_00N_100E.tif', '../../06_data/deforestation/global_forest_watch/Hansen_GFC_2021_v1_9_lossyear_00N_110E.tif', '../../06_data/deforestation/global_forest_watch/Hansen_GFC_2021_v1_9_lossyear_10N_100E.tif', '../../06_data/deforestation/global_forest_watch/Hansen_GFC_2021_v1_9_lossyear_10N_110E.tif']

merge_tiff_files(tif_files, '../../07_data_prepared/deforestation/gfw_lossyear_2000_2022_borneo.tif')

clip_tif_to_bbox('../../07_data_prepared/deforestation/gfw_lossear_2000_2022_borneo.tif', gdf_boundary, '../../07_data_prepared/deforestation/gfw_2000_2022_borneo.tif')

change_values_outside_mask('../../07_data_prepared/deforestation/gfw_lossyear_2000_2022_borneo.tif', '../../07_data_prepared/deforestation/gfw_2000_2022_borneo.tif', gdf_boundary, 0)

# reproject to epsg:23846
reproject_raster('EPSG:23846', '../../07_data_prepared/deforestation/gfw_lossyear_2000_2022_borneo.tif', '../../07_data_prepared/deforestation/utm_gfw_2000_2022_borneo.tif')

## forest cover 2000


In [ ]:
tif_files = ['../../06_data/deforestation/global_forest_watch/Hansen_GFC_2021_v1_9_treecover2000_00N_100E.tif', '../../06_data/deforestation/global_forest_watch/Hansen_GFC_2021_v1_9_treecover2000_00N_110E.tif', '../../06_data/deforestation/global_forest_watch/Hansen_GFC_2021_v1_9_treecover2000_10N_100E.tif', '../../06_data/deforestation/global_forest_watch/Hansen_GFC_2021_v1_9_treecover2000_10N_110E.tif']

merge_tiff_files(tif_files, '../../07_data_prepared/deforestation/gfw_fcover_2000_borneo.tif')

clip_tif_to_bbox('../../07_data_prepared/deforestation/gfw_fcover_2000_borneo.tif', gdf_boundary, '../../07_data_prepared/deforestation/gfw_fcover_2000_borneo.tif')

change_values_outside_mask('../../07_data_prepared/deforestation/gfw_fcover_2000_borneo.tif', '../../07_data_prepared/deforestation/gfw_fcover_2000_borneo.tif', gdf_boundary, 0)

# reproject to epsg:23846
reproject_raster('EPSG:23846', '../../07_data_prepared/deforestation/gfw_fcover_2000_borneo.tif', '../../07_data_prepared/deforestation/utm_gfw_fcover_2000_borneo.tif')

## forest gain 


In [ ]:
tif_files = ['../../06_data/deforestation/global_forest_watch/Hansen_GFC_2021_v1_9_gain_00N_100E.tif', '../../06_data/deforestation/global_forest_watch/Hansen_GFC_2021_v1_9_gain_00N_110E.tif', '../../06_data/deforestation/global_forest_watch/Hansen_GFC_2021_v1_9_gain_10N_100E.tif', '../../06_data/deforestation/global_forest_watch/Hansen_GFC_2021_v1_9_gain_10N_110E.tif']

merge_tiff_files(tif_files, '../../07_data_prepared/deforestation/gfw_gain_2000_2012_borneo.tif')

clip_tif_to_bbox('../../07_data_prepared/deforestation/gfw_gain_2000_2012_borneo.tif', gdf_boundary, '../../07_data_prepared/deforestation/gfw_gain_2000_2012_borneo.tif')

change_values_outside_mask('../../07_data_prepared/deforestation/gfw_gain_2000_2012_borneo.tif', '../../07_data_prepared/deforestation/gfw_gain_2000_2012_borneo.tif', gdf_boundary, 0)

# reproject to epsg:23846
reproject_raster('EPSG:23846', '../../07_data_prepared/deforestation/gfw_gain_2000_2012_borneo.tif', '../../07_data_prepared/deforestation/utm_gfw_gain_2000_2012_borneo_test.tif')

# prepare forest fire data


In [ ]:
clip_tif_to_bbox('../../06_data/deforestation/forest_fires/fire_forest_loss_2001-22_annual.tiff', gdf_boundary, '../../07_data_prepared/deforestation/forest_fires.tif')

change_values_outside_mask('../../07_data_prepared/deforestation/forest_fires.tif', '../../07_data_prepared/deforestation/forest_fires.tif', gdf_boundary, 0)

# reproject to epsg:23846
reproject_raster('EPSG:23846', '../../07_data_prepared/deforestation/forest_fires.tif', '../../07_data_prepared/deforestation/utm_forest_fires.tif')

# primary forest


In [ ]:
clip_tif_to_bbox('../../06_data/deforestation/primary_forest/Asia_2001_primary.tif', gdf_boundary, '../../07_data_prepared/deforestation/primary_forest_2001.tif')

change_values_outside_mask('../../07_data_prepared/deforestation/primary_forest_2001.tif', '../../07_data_prepared/deforestation/primary_forest_2001.tif', gdf_boundary, 0)

# reproject to epsg:23846
reproject_raster('EPSG:23846', '../../07_data_prepared/deforestation/primary_forest_2001.tif', '../../07_data_prepared/deforestation/utm_primary_forest_2001.tif')

# clip protected areas to boundary


In [ ]:
shapefile_path = "../../06_data/protected_areas/WDPA_WDOECM_Apr2023_Public_AS_shp/WDPA_WDOECM_Apr2023_Public_AS_shp_0/WDPA_WDOECM_Apr2023_Public_AS_shp-polygons.shp"
shapefile_gdf = gpd.read_file(shapefile_path)

gdf_boundary = ox.geocode_to_gdf("borneo").dropna(axis=1, how="any")

clipped_gdf = gpd.clip(shapefile_gdf, gdf_boundary.geometry)

clipped_gdf = clipped_gdf.to_crs("EPSG:23846")

output_path = "../../07_data_prepared/protected_areas/protected_areas_borneo.shp"
clipped_gdf.to_file(output_path)

# RSPO concessions


In [ ]:
shapefile_path = "../../06_data/RSPO/RSPO_Concessions_Versions6_20Jan2023.shp"
shapefile_gdf = gpd.read_file(shapefile_path)
shapefile_gdf = shapefile_gdf.to_crs(epsg=4326)

gdf_boundary = ox.geocode_to_gdf("borneo").dropna(axis=1, how="any")

clipped_gdf = gpd.clip(shapefile_gdf, gdf_boundary.geometry)

clipped_gdf = clipped_gdf.to_crs("EPSG:23846")

output_path = "../../07_data_prepared/RSPO/utm_RSPO_2023.shp"
clipped_gdf.to_file(output_path)

# prepare cropland data


In [ ]:
import os
import glob

# Save the current working directory
script_dir = os.path.abspath(os.path.dirname(__file__))

# Set the cropland directory
cropland_dir = os.path.join(script_dir, '../../06_data/vegetation/cropland')

# Get a list of all TIFF files in the cropland directory
tif_files = [os.path.abspath(file) for file in glob.glob(os.path.join(cropland_dir, '*.tif'))]

# Set the output directory for the clipped TIFF files
clipped_cropland_dir = os.path.join(script_dir, '../../06_data/vegetation/cropland/clipped_borneo')
os.makedirs(clipped_cropland_dir, exist_ok=True)

# Clip the TIFF files
for file in tif_files:
    # Construct the output file name
    output_file = os.path.basename(file).replace('Global', 'borneo')

    # Set the output file path
    output_path = os.path.join(clipped_cropland_dir, output_file)

    # Clip the TIFF file
    clip_tif_to_bbox(file, gdf_boundary, output_path)

tif_files_2003 = [os.path.join(clipped_cropland_dir,  'borneo_cropland_NE_2003.tif'), os.path.join(clipped_cropland_dir,  'borneo_cropland_SE_2003.tif')]
tif_files_2007 = [os.path.join(clipped_cropland_dir,  'borneo_cropland_NE_2007.tif'), os.path.join(clipped_cropland_dir,  'borneo_cropland_SE_2007.tif')]
tif_files_2011 = [os.path.join(clipped_cropland_dir,  'borneo_cropland_NE_2011.tif'), os.path.join(clipped_cropland_dir,  'borneo_cropland_SE_2011.tif')]
tif_files_2015 = [os.path.join(clipped_cropland_dir,  'borneo_cropland_NE_2015.tif'), os.path.join(clipped_cropland_dir,  'borneo_cropland_SE_2015.tif')]
tif_files_2019 = [os.path.join(clipped_cropland_dir,  'borneo_cropland_NE_2019.tif'), os.path.join(clipped_cropland_dir,  'borneo_cropland_SE_2019.tif')]

print(tif_files_2003)
# Set the output directory for the merged TIFF files
merge_output_dir = os.path.join(script_dir, '../../07_data_prepared/cropland')
os.makedirs(merge_output_dir, exist_ok=True)

merge_tiff_files(tif_files_2003, os.path.join(merge_output_dir,f'borneo_cropland_2003.tif'))
merge_tiff_files(tif_files_2007, os.path.join(merge_output_dir,f'borneo_cropland_2007.tif'))
merge_tiff_files(tif_files_2011, os.path.join(merge_output_dir,f'borneo_cropland_2011.tif'))
merge_tiff_files(tif_files_2015, os.path.join(merge_output_dir,f'borneo_cropland_2015.tif'))
merge_tiff_files(tif_files_2019, os.path.join(merge_output_dir,f'borneo_cropland_2019.tif'))

change_values_outside_mask(os.path.join(merge_output_dir,f'borneo_cropland_2003.tif'), os.path.join(merge_output_dir,f'borneo_cropland_2003.tif'), mask=gdf_boundary)
change_values_outside_mask(os.path.join(merge_output_dir,f'borneo_cropland_2007.tif'), os.path.join(merge_output_dir,f'borneo_cropland_2007.tif'), mask=gdf_boundary)
change_values_outside_mask(os.path.join(merge_output_dir,f'borneo_cropland_2011.tif'), os.path.join(merge_output_dir,f'borneo_cropland_2011.tif'), mask=gdf_boundary)
change_values_outside_mask(os.path.join(merge_output_dir,f'borneo_cropland_2015.tif'), os.path.join(merge_output_dir,f'borneo_cropland_2015.tif'), mask=gdf_boundary)
change_values_outside_mask(os.path.join(merge_output_dir,f'borneo_cropland_2019.tif'), os.path.join(merge_output_dir,f'borneo_cropland_2019.tif'), mask=gdf_boundary)

reproject_raster("EPSG:23846", os.path.join(merge_output_dir,f'borneo_cropland_2003.tif'), os.path.join(merge_output_dir,f'utm_borneo_cropland_2003.tif'))
reproject_raster("EPSG:23846", os.path.join(merge_output_dir,f'borneo_cropland_2007.tif'), os.path.join(merge_output_dir,f'utm_borneo_cropland_2007.tif'))
reproject_raster("EPSG:23846", os.path.join(merge_output_dir,f'borneo_cropland_2011.tif'), os.path.join(merge_output_dir,f'utm_borneo_cropland_2011.tif'))
reproject_raster("EPSG:23846", os.path.join(merge_output_dir,f'borneo_cropland_2015.tif'), os.path.join(merge_output_dir,f'utm_borneo_cropland_2015.tif'))
reproject_raster("EPSG:23846", os.path.join(merge_output_dir,f'borneo_cropland_2019.tif'), os.path.join(merge_output_dir,f'utm_borneo_cropland_2019.tif'))


# prepare infrastructure data


In [ ]:
import os
import glob

# Save the current working directory
script_dir = os.path.abspath(os.path.dirname(__file__))

# Set the cropland directory
infra_dir = os.path.join(script_dir, '../../06_data/infrastructure/built_up_areas_2000_2020')

# Get a list of all TIFF files in the cropland directory
tif_files = [os.path.abspath(file) for file in glob.glob(os.path.join(infra_dir, '*.tif'))]

# Set the output directory for the clipped TIFF files
clipped__dir = os.path.join(script_dir, '../../06_data/infrastructure/built_up_areas_2000_2020/borneo_clipped')
os.makedirs(infra_dir, exist_ok=True)

# Clip the TIFF files
for file in tif_files:
    # Construct the output file name
    output_file = os.path.basename(file).replace('Change', 'Borneo_Change')

    # Set the output file path
    output_path = os.path.join(clipped__dir, output_file)

    # Clip the TIFF file
    clip_tif_to_bbox(file, gdf_boundary, output_path)

tif_files = [os.path.abspath(file) for file in glob.glob(os.path.join(clipped__dir, '*.tif'))]

# Set the output directory for the merged TIFF files
merge_output_dir = os.path.join(script_dir, '../../07_data_prepared/infrastructure')
os.makedirs(merge_output_dir, exist_ok=True)

merge_tiff_files(tif_files, os.path.join(merge_output_dir,f'build_up_area.tif'))


change_values_outside_mask(os.path.join(merge_output_dir,f'build_up_area.tif'), os.path.join(merge_output_dir,f'build_up_area.tif'), mask=gdf_boundary)


reproject_raster("EPSG:23846", os.path.join(merge_output_dir,f'build_up_area.tif'), os.path.join(merge_output_dir,f'utm_build_up_area.tif'))
